In [68]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2018, day=7)

def parses(input):
    vals = []
    for line in input.strip().split('\n'):
        vs = parse.parse('Step {} must be finished before step {} can begin.', line).fixed
        vals.append((vs[0], vs[1]))
    return vals

data = parses(puzzle.input_data)

In [114]:
puzzle.easter_eggs

[<span title="Just some oak and some pine and a handful of Norsemen.">Some assembly required.</span>]

In [69]:
data

[('A', 'I'),
 ('M', 'Q'),
 ('B', 'S'),
 ('G', 'N'),
 ('Y', 'R'),
 ('E', 'H'),
 ('K', 'L'),
 ('H', 'Z'),
 ('C', 'P'),
 ('W', 'U'),
 ('V', 'L'),
 ('O', 'N'),
 ('U', 'I'),
 ('D', 'P'),
 ('Q', 'L'),
 ('F', 'Z'),
 ('L', 'N'),
 ('P', 'S'),
 ('I', 'S'),
 ('S', 'R'),
 ('T', 'N'),
 ('N', 'X'),
 ('Z', 'J'),
 ('R', 'J'),
 ('J', 'X'),
 ('E', 'I'),
 ('T', 'R'),
 ('I', 'N'),
 ('K', 'C'),
 ('B', 'D'),
 ('K', 'T'),
 ('E', 'P'),
 ('F', 'I'),
 ('O', 'U'),
 ('I', 'J'),
 ('S', 'Z'),
 ('L', 'J'),
 ('F', 'T'),
 ('F', 'P'),
 ('I', 'T'),
 ('G', 'S'),
 ('V', 'U'),
 ('F', 'R'),
 ('L', 'R'),
 ('Y', 'D'),
 ('M', 'E'),
 ('U', 'L'),
 ('C', 'D'),
 ('W', 'N'),
 ('S', 'N'),
 ('O', 'S'),
 ('B', 'T'),
 ('V', 'T'),
 ('S', 'X'),
 ('V', 'P'),
 ('F', 'L'),
 ('P', 'R'),
 ('D', 'N'),
 ('C', 'L'),
 ('O', 'Q'),
 ('N', 'Z'),
 ('Y', 'L'),
 ('B', 'K'),
 ('P', 'Z'),
 ('V', 'Z'),
 ('U', 'J'),
 ('Q', 'S'),
 ('H', 'F'),
 ('E', 'O'),
 ('D', 'F'),
 ('D', 'X'),
 ('L', 'S'),
 ('Z', 'R'),
 ('K', 'X'),
 ('M', 'V'),
 ('A', 'M'),
 ('B', 'W'),

In [70]:
sample = parses("""Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
""")

In [71]:
sample

[('C', 'A'),
 ('C', 'F'),
 ('A', 'B'),
 ('A', 'D'),
 ('B', 'E'),
 ('D', 'E'),
 ('F', 'E')]

In [72]:
def topsort(steps):
    graph = defaultdict(list)
    n_incoming = defaultdict(int)
    for before, after in steps:
        graph[before].append(after)
        n_incoming[after] += 1
        n_incoming[before]
    no_incoming = sorted([k for k, v in n_incoming.items() if v == 0])
    
    top_order = []
    while len(no_incoming) > 0:
        node = no_incoming.pop(0)
        top_order.append(node)
        for child in graph[node]:
            n_incoming[child] -= 1
            if n_incoming[child] == 0:
                no_incoming.append(child)
        no_incoming.sort()
    return "".join(top_order)

In [73]:
assert topsort(sample) == 'CABDFE'

In [74]:
topsort(data)

'ABGKCMVWYDEHFOPQUILSTNZRJX'

In [75]:
puzzle.answer_a = topsort(data)

In [80]:
from networkx import DiGraph, lexicographical_topological_sort as lt_sort

"".join(lt_sort(DiGraph((data))))

'ABGKCMVWYDEHFOPQUILSTNZRJX'

In [110]:
def mintime(steps, maxworkers=5, baseT=60):
    graph = defaultdict(list)
    n_incoming = defaultdict(int)
    for before, after in steps:
        graph[before].append(after)
        n_incoming[after] += 1
        n_incoming[before]
    ready = sorted([k for k, v in n_incoming.items() if v == 0])
    times = {k: ord(k)-ord('A')+1+baseT for k in n_incoming}

    time = 0
    completed = 0
    workers = [(None, None) for _ in range(maxworkers)]
    
    while True:
        for i, (task, eta) in enumerate(workers):
            if task is not None and (eta == time):
                completed += 1
                for child in graph[task]:
                    n_incoming[child] -= 1
                    if n_incoming[child] == 0:
                        ready.append(child)
                        ready.sort()
                workers[i] = (None, None)

        for i, (task, eta) in enumerate(workers):            
            if task is None and len(ready) > 0:
                task = ready.pop(0)
                eta = time + times[task]
                workers[i] = (task, eta)
        if completed == len(times):
            break
        time = min(eta for _, eta in workers if eta)
        
    return time

In [111]:
assert mintime(sample, 2, 0) == 15

In [112]:
mintime(data)

898

In [113]:
puzzle.answer_b = mintime(data)